### Handle ERA5 land dataset with Zampy
Demo notebook for developers.

In [1]:
import numpy as np
from zampy.datasets import ERA5Land
from zampy.datasets.dataset_protocol import TimeBounds, SpatialBounds
from pathlib import Path

/home/yangliu/mambaforge/envs/ecoextreml/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
work_dir = Path("/home/yangliu/EcoExtreML/temp")
download_dir = work_dir / "download"
ingest_dir = work_dir / "ingest"
times = TimeBounds(np.datetime64("2010-01-01T00:00:00"), np.datetime64("2010-01-31T23:00:00"))
bbox_demo = SpatialBounds(54, 56, 1, 3)

Download dataset.

In [3]:
era5_land_dataset = ERA5Land()
era5_land_dataset.download(
    download_dir=download_dir,
    time_bounds=times,
    spatial_bounds=bbox_demo,
    variable_names=["2m_temperature", "2m_dewpoint_temperature"],
)

 50%|█████     | 1/2 [00:00<00:00,  1.63it/s]

File 'era5-land_2m_temperature_2010-1.nc' already exists, skipping...


100%|██████████| 2/2 [00:00<00:00,  2.01it/s]

File 'era5-land_2m_dewpoint_temperature_2010-1.nc' already exists, skipping...


True

Data ingestion to the unified format in `zampy`.

In [4]:
era5_land_dataset.ingest(download_dir, ingest_dir)

True

In [7]:
ds = era5_land_dataset.load(
    ingest_dir=ingest_dir,
    time_bounds=times,
    spatial_bounds=bbox_demo,
    variable_names=["2m_temperature", "2m_dewpoint_temperature"],
    resolution=1.0,
    regrid_method="flox",
)

In [8]:
ds

<xarray.Dataset>
Dimensions:          (time: 744, latitude: 54, longitude: 54)
Coordinates:
  * time             (time) datetime64[ns] 2010-01-01 ... 2010-01-31T23:00:00
  * latitude         (latitude) float64 1.0 2.0 3.0 4.0 ... 51.0 52.0 53.0 54.0
  * longitude        (longitude) float64 3.0 4.0 5.0 6.0 ... 53.0 54.0 55.0 56.0
Data variables:
    air_temperature  (time, latitude, longitude) float32 dask.array<chunksize=(744, 54, 54), meta=np.ndarray>
    d2m              (time, latitude, longitude) float32 dask.array<chunksize=(744, 54, 54), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2023-08-08 08:58:46 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [9]:
from zampy.datasets import converter

ds_convert = converter.convert(ds, era5_land_dataset, "ALMA")

air_temperature renamed to Tair.
Variable 'd2m' is not included in 'ALMA' convention.
Conversion of dataset 'era5-land' following ALMA convention is complete!


In [10]:
ds_convert

<xarray.Dataset>
Dimensions:    (time: 744, latitude: 54, longitude: 54)
Coordinates:
  * time       (time) datetime64[ns] 2010-01-01 ... 2010-01-31T23:00:00
  * latitude   (latitude) float64 1.0 2.0 3.0 4.0 5.0 ... 51.0 52.0 53.0 54.0
  * longitude  (longitude) float64 3.0 4.0 5.0 6.0 7.0 ... 53.0 54.0 55.0 56.0
Data variables:
    Tair       (time, latitude, longitude) float32 dask.array<chunksize=(744, 54, 54), meta=np.ndarray>
    d2m        (time, latitude, longitude) float32 dask.array<chunksize=(744, 54, 54), meta=np.ndarray>
Attributes:
    Conventions:  ALMA
    history:      2023-08-08 08:58:46 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...